# Automate autoReduce (12)
In this notebook, we will try to automate the sbml to ode process for autoReduce. <br>
7.14.2020

In [1]:
# imports
from libsbml import *
import sys
import numpy as np
from auto_reduce import *
from auto_reduce.utils import get_ODE
import matplotlib.pyplot as plt
from auto_reduce.utils import reduce

**What do we want?** <br>
- set num species
- set num params
- get the value for each param
- set P = np.zeros(num_params)
- set P[i] = a parameter
- params_values = P.copy()
- define timepoints_ode
- define error_tol, nstates_tol
- define initial conditions
- x,f,P = system.load_ODE_model(n,num+params)
- Define species, x[i] = whatever
- define reactions
- for each species define a function f[i] = ad;sklfj
- setup output matrix
- sys = System()
- sys_ode - getODE

In [45]:
filename = 'CRN.xml'
t0 = 0
tend = 100
numPoints = 100

doc = readSBMLFromFile(filename)
if doc.getNumErrors(LIBSBML_SEV_FATAL):
  print('Encountered serious errors while reading file')
  print(doc.getErrorLog().toString())
  sys.exit(1)
  
# clear errors
doc.getErrorLog().clearLog()
  
  #

props = ConversionProperties()
props.addOption("promoteLocalParameters", True)
  
if doc.convert(props) != LIBSBML_OPERATION_SUCCESS: 
  print('The document could not be converted')
  print(doc.getErrorLog().toString())
  
props = ConversionProperties()
props.addOption("expandInitialAssignments", True)

if doc.convert(props) != LIBSBML_OPERATION_SUCCESS: 
  print('The document could not be converted')
  print(doc.getErrorLog().toString())
  
props = ConversionProperties()
props.addOption("expandFunctionDefinitions", True)

if doc.convert(props) != LIBSBML_OPERATION_SUCCESS: 
  print('The document could not be converted')
  print(doc.getErrorLog().toString())
  
#
# figure out which species are variable 
#
mod = doc.getModel()

In [47]:
num_species = mod.getNumSpecies()
num_params = mod.getNumParameters() # DOESNt WORK??

In [48]:
num_params

80

In [82]:
species = mod.getSpecies(1)
species.getBoundaryCondition() == True

False

In [94]:
# Get a species list
initial_conditions = {}
variables = {}
for i in range(num_species):
    
    species = mod.getSpecies(i)
    variables[species.getId()] = [] # fills out dictionary of species with []
    
    # IGNORE BOUNDARY SPECIES
#     if species.getBoundaryCondition() == False:
#         continue
#     if species.isSetInitialConcentration():
#         print(here)
#         initial_conditions['species.getId()'] = species.getInitialConcentration()
#         print(initial_conditions)
#     else:
#         initial_conditions[species.getId()] = species.getInitialAmount()

In [95]:
# Get a parameters dictionary
parameters = {}
for i in range(num_params):
    element = mod.getParameter(i)
   # print(element.getId())
    parameters['{e}'.format(e = element.getId())] = element.getValue() # fills out dictionary of parameters with k

In [104]:
# Set all parameters equal to a P[i]
P = np.zeros(num_params)
p_values = list(parameters.values())

for i in range(num_params):
    P[i] = p_values[i]
    
# Autoreduce setup stuff
params_values = P.copy()
timepoints_ode = np.linspace(t0,tend,numPoints)
error_tol = 100
nstates_tol = 1
x,f,P = system.load_ODE_model(num_species, num_params)
params = P

Get functions :0<br>
Instead of doing enzyme_hex = x[0], and referring by enzyme_hex, just refer to by x[i] everywhere.

In [114]:
# Get individual reactions
reactions = {}
num_reactions = mod.getNumReactions()

for i in range(num_reactions):
    reaction = mod.getReaction(i)
    kinetics = reaction.getKineticLaw()
    reactions[reaction.getId()] = kinetics.getFormula()

In [116]:
reactions.keys

{'r0': 'r0_k * enzyme_hex * metabolite_atp * molecule_glucose',
 'r0_1': 'r0_1_k * complex_enzyme_hex_metabolite_atp_molecule_glucose',
 'r2': 'r2_k * complex_enzyme_hex_metabolite_atp_molecule_glucose',
 'r3': 'r3_k * complex_enzyme_hex_metabolite_adp_molecule_g6p',
 'r3_1': 'r3_1_k * enzyme_hex * molecule_g6p * metabolite_adp',
 'r5': 'r5_k * enzyme_pgi * molecule_g6p',
 'r5_1': 'r5_1_k * complex_enzyme_pgi_molecule_g6p',
 'r7': 'r7_k * complex_enzyme_pgi_molecule_g6p',
 'r8': 'r8_k * complex_enzyme_pgi_molecule_f6p',
 'r8_1': 'r8_1_k * enzyme_pgi * molecule_f6p',
 'r10': 'r10_k * enzyme_pfk * metabolite_atp * molecule_f6p',
 'r10_1': 'r10_1_k * complex_enzyme_pfk_metabolite_atp_molecule_f6p',
 'r12': 'r12_k * complex_enzyme_pfk_metabolite_atp_molecule_f6p',
 'r13': 'r13_k * complex_enzyme_pfk_metabolite_adp_molecule_f16p',
 'r13_1': 'r13_1_k * enzyme_pfk * molecule_f16p * metabolite_adp',
 'r15': 'r15_k * enzyme_ald_tpi * molecule_f16p',
 'r15_1': 'r15_1_k * complex_enzyme_ald_tpi_m

In [139]:
x[0]

x0

In [147]:
def generateCodeForFile(filename,  t0=0, tEnd=10, numPoints=1000):
  # 
  # read the SBML from file 
  # 
  doc = readSBMLFromFile(filename)
  if doc.getNumErrors(LIBSBML_SEV_FATAL):
    print('Encountered serious errors while reading file')
    print(doc.getErrorLog().toString())
    sys.exit(1)
    
  # clear errors
  doc.getErrorLog().clearLog()
  
  #
  # perform conversions
  #
  props = ConversionProperties()
  props.addOption("promoteLocalParameters", True)
  
  if doc.convert(props) != LIBSBML_OPERATION_SUCCESS: 
    print('The document could not be converted')
    print(doc.getErrorLog().toString())
    
  props = ConversionProperties()
  props.addOption("expandInitialAssignments", True)
  
  if doc.convert(props) != LIBSBML_OPERATION_SUCCESS: 
    print('The document could not be converted')
    print(doc.getErrorLog().toString())
    
  props = ConversionProperties()
  props.addOption("expandFunctionDefinitions", True)
  
  if doc.convert(props) != LIBSBML_OPERATION_SUCCESS: 
    print('The document could not be converted')
    print(doc.getErrorLog().toString())
    
  #
  # figure out which species are variable 
  #
  mod = doc.getModel()
  variables = {}
#  print('params', mod.getNumParameters())

  for i in range(mod.getNumSpecies()): 
     species = mod.getSpecies(i)
    # if species.getBoundaryCondition() == True or variables.has_key(species.getId()):
     if species.getBoundaryCondition() == True or (species.getId() in variables.keys()):


       continue
     variables[species.getId()] = []
  #print(variables)
  
  #
  # start generating the code by appending to bytearray
  #
  generated_code = bytearray()
  generated_code.extend('from numpy import *\n'.encode('latin-1'))
  generated_code.extend('x,f,P = system.load_ODE_model(num_species, num_params)'.encode('latin-1'))


 # generated_code.extend('from matplotlib.pylab import *\n'.encode('latin-1'))
 # generated_code.extend('from matplotlib.pyplot import *\n'.encode('latin-1'))
 # generated_code.extend('from scipy.integrate import odeint \n'.encode('latin-1'))
  
  generated_code.extend('\n'.encode('latin-1'))
#  generated_code.extend('def simulateModel(t0, tend, numPoints):\n'.encode('latin-1'))
  
  # write out compartment values 
  generated_code.extend('\n'.encode('latin-1'))
  generated_code.extend('#compartments\n'.encode('latin-1'))
  for i in range(mod.getNumCompartments()):
    element = mod.getCompartment(i)
    generated_code.extend('{0} = {1}\n'.format(element.getId(), element.getSize()).encode('latin-1'))
  
  # write out parameter values 
  generated_code.extend('\n'.encode('latin-1'))
  generated_code.extend('#global parameters\n'.encode('latin-1'))
  for i in range(mod.getNumParameters()):
    element = mod.getParameter(i)
    generated_code.extend('{0} = {1}\n'.format(element.getId(), element.getValue()).encode('latin-1'))
  
  # write out boundary species 
  generated_code.extend('\n'.encode('latin-1'))
  generated_code.extend('#boundary species\n'.encode('latin-1'))
  for i in range(mod.getNumSpecies()):
    element = mod.getSpecies(i)
    if element.getBoundaryCondition() == False: 
      continue 
    if element.isSetInitialConcentration(): 
      generated_code.extend('{0} = {1}\n'.format(element.getId(), element.getInitialConcentration()).encode('latin-1'))
    else:
      generated_code.extend('{0} = {1}\n'.format(element.getId(), element.getInitialAmount()).encode('latin-1'))  
  
  # write derive function
  generated_code.extend('\n'.encode('latin-1'))
  #generated_code.extend('  def ode_fun(__Y__, t):\n'.encode('latin-1'))
    
   # Set all species equal to a __Y__[i] 
  for i in range(len(variables.keys())): 
    generated_code.extend('{0} = x[{1}]\n'.format(list(variables.keys())[i], i).encode('latin-1'))
    
  generated_code.extend('\n'.encode('latin-1'))
  
  for i in range(mod.getNumReactions()): 
    reaction = mod.getReaction(i)
    kinetics = reaction.getKineticLaw()  
    
    generated_code.extend('{0} = {1}\n'.format(reaction.getId(),  kinetics.getFormula()).encode('latin-1'))
    
    for j in range(reaction.getNumReactants()): 
      ref = reaction.getReactant(j)
      species = mod.getSpecies(ref.getSpecies())
      if species.getBoundaryCondition() == True: 
        continue
      if ref.getStoichiometry() == 1.0: 
        variables[species.getId()].append('-{0}'.format(reaction.getId()))
      else:
        variables[species.getId()].append('-({0})*{1}'.format(ref.getStoichiometry(), reaction.getId()))
    for j in range(reaction.getNumProducts()): 
      ref = reaction.getProduct(j)
      species = mod.getSpecies(ref.getSpecies())
      if species.getBoundaryCondition() == True: 
        continue
      if ref.getStoichiometry() == 1.0: 
        variables[species.getId()].append('{0}'.format(reaction.getId()))
      else:
        variables[species.getId()].append('({0})*{1}'.format(ref.getStoichiometry(), reaction.getId()))
  
  generated_code.extend('\n'.encode('latin-1'))
    
  generated_code.extend('funcs = array(['.encode('latin-1'))
 # generated_code.extend('    print( array(['.encode('latin-1'))
  for i in range(len(variables.keys())):
    for eqn in variables[list(variables.keys())[i]]:
      generated_code.extend(' + ({0})'.format(eqn).encode('latin-1'))
    if i + 1 < len(variables.keys()):
      generated_code.extend(',\n      '.encode('latin-1'))
  generated_code.extend('    ])\n'.encode('latin-1'))
  generated_code.extend('\n'.encode('latin-1'))

  return generated_code


  

In [149]:
gen_code = generateCodeForFile("CRN.xml")
exec(gen_code)
funcs[0]

-15.0*x0*x1*x2 - 1.5*x0*x3*x4 + 1.5*x5 + 15.0*x6

In [145]:
gen_code

bytearray(b'from numpy import *\nx,f,P = system.load_ODE_model(num_species, num_params\n\n#compartments\ndefault = 1e-06\n\n#global parameters\nr0_k = 15.0\nr0_1_k = 1.5\nr2_k = 36000.0\nr3_k = 15.0\nr3_1_k = 1.5\nr5_k = 15.0\nr5_1_k = 1.5\nr7_k = 36000.0\nr8_k = 15.0\nr8_1_k = 1.5\nr10_k = 15.0\nr10_1_k = 1.5\nr12_k = 36000.0\nr13_k = 15.0\nr13_1_k = 1.5\nr15_k = 15.0\nr15_1_k = 1.5\nr17_k = 36000.0\nr18_k = 15.0\nr18_1_k = 1.5\nr20_k = 15.0\nr20_1_k = 1.5\nr22_k = 36000.0\nr23_k = 15.0\nr23_1_k = 1.5\nr25_k = 15.0\nr25_1_k = 1.5\nr27_k = 36000.0\nr28_k = 15.0\nr28_1_k = 1.5\nr30_k = 15.0\nr30_1_k = 1.5\nr32_k = 36000.0\nr33_k = 15.0\nr33_1_k = 1.5\nr35_k = 15.0\nr35_1_k = 1.5\nr37_k = 36000.0\nr38_k = 15.0\nr38_1_k = 1.5\nr40_k = 15.0\nr40_1_k = 1.5\nr42_k = 36000.0\nr43_k = 15.0\nr43_1_k = 1.5\nr45_k = 15.0\nr45_1_k = 1.5\nr47_k = 36000.0\nr48_k = 15.0\nr48_1_k = 1.5\nr50_k = 15.0\nr50_1_k = 1.5\nr52_k = 36000.0\nr53_k = 15.0\nr53_1_k = 1.5\nr55_k = 15.0\nr55_1_k = 1.5\nr57_k = 3600

In [146]:
print('from numpy import *\nx,f,P = system.load_ODE_model(num_species, num_params\n\n#compartments\ndefault = 1e-06\n\n#global parameters\nr0_k = 15.0\nr0_1_k = 1.5\nr2_k = 36000.0\nr3_k = 15.0\nr3_1_k = 1.5\nr5_k = 15.0\nr5_1_k = 1.5\nr7_k = 36000.0\nr8_k = 15.0\nr8_1_k = 1.5\nr10_k = 15.0\nr10_1_k = 1.5\nr12_k = 36000.0\nr13_k = 15.0\nr13_1_k = 1.5\nr15_k = 15.0\nr15_1_k = 1.5\nr17_k = 36000.0\nr18_k = 15.0\nr18_1_k = 1.5\nr20_k = 15.0\nr20_1_k = 1.5\nr22_k = 36000.0\nr23_k = 15.0\nr23_1_k = 1.5\nr25_k = 15.0\nr25_1_k = 1.5\nr27_k = 36000.0\nr28_k = 15.0\nr28_1_k = 1.5\nr30_k = 15.0\nr30_1_k = 1.5\nr32_k = 36000.0\nr33_k = 15.0\nr33_1_k = 1.5\nr35_k = 15.0\nr35_1_k = 1.5\nr37_k = 36000.0\nr38_k = 15.0\nr38_1_k = 1.5\nr40_k = 15.0\nr40_1_k = 1.5\nr42_k = 36000.0\nr43_k = 15.0\nr43_1_k = 1.5\nr45_k = 15.0\nr45_1_k = 1.5\nr47_k = 36000.0\nr48_k = 15.0\nr48_1_k = 1.5\nr50_k = 15.0\nr50_1_k = 1.5\nr52_k = 36000.0\nr53_k = 15.0\nr53_1_k = 1.5\nr55_k = 15.0\nr55_1_k = 1.5\nr57_k = 36000.0\nr58_k = 15.0\nr58_1_k = 1.5\nr60_k = 15.0\nr60_1_k = 1.5\nr62_k = 36000.0\nr63_k = 15.0\nr63_1_k = 1.5\nr65_k = 15.0\nr65_1_k = 1.5\nr67_k = 36000.0\nr68_k = 15.0\nr68_1_k = 1.5\nr70_k = 15.0\nr70_1_k = 1.5\nr72_k = 36000.0\nr73_k = 15.0\nr73_1_k = 1.5\nr75_k = 15.0\nr75_1_k = 1.5\nr77_k = 2.0\nr78_k = 15.0\nr78_1_k = 1.5\n\n#boundary species\n\nenzyme_hex = x[0]\nmetabolite_atp = x[1]\nmolecule_glucose = x[2]\nmolecule_g6p = x[3]\nmetabolite_adp = x[4]\ncomplex_enzyme_hex_metabolite_atp_molecule_glucose = x[5]\ncomplex_enzyme_hex_metabolite_adp_molecule_g6p = x[6]\nenzyme_pgi = x[7]\nmolecule_f6p = x[8]\ncomplex_enzyme_pgi_molecule_g6p = x[9]\ncomplex_enzyme_pgi_molecule_f6p = x[10]\nenzyme_pfk = x[11]\nmolecule_f16p = x[12]\ncomplex_enzyme_pfk_metabolite_atp_molecule_f6p = x[13]\ncomplex_enzyme_pfk_metabolite_adp_molecule_f16p = x[14]\nenzyme_ald_tpi = x[15]\nmolecule_g3p = x[16]\ncomplex_enzyme_ald_tpi_molecule_f16p = x[17]\ncomplex_enzyme_ald_tpi_2x_molecule_g3p = x[18]\nenzyme_gapN = x[19]\nmetabolite_nadp = x[20]\nmolecule_3pg = x[21]\nmetabolite_nadph = x[22]\ncomplex_enzyme_gapN_2x_metabolite_nadp_2x_molecule_g3p = x[23]\ncomplex_enzyme_gapN_2x_metabolite_nadph_2x_molecule_3pg = x[24]\nenzyme_mGapDH = x[25]\nmetabolite_pi = x[26]\nmolecule_13bpg = x[27]\ncomplex_enzyme_mGapDH_2x_metabolite_nadp_metabolite_pi_2x_molecule_g3p = x[28]\ncomplex_enzyme_mGapDH_2x_metabolite_nadph_molecule_13bpg = x[29]\nenzyme_pgk = x[30]\ncomplex_enzyme_pgk_metabolite_adp_molecule_13bpg = x[31]\ncomplex_enzyme_pgk_metabolite_atp_2x_molecule_3pg = x[32]\nenzyme_pgm = x[33]\nmolecule_2pg = x[34]\ncomplex_enzyme_pgm_2x_molecule_3pg = x[35]\ncomplex_enzyme_pgm_2x_molecule_2pg = x[36]\nenzyme_eno = x[37]\nmolecule_pep = x[38]\ncomplex_enzyme_eno_2x_molecule_2pg = x[39]\ncomplex_enzyme_eno_2x_molecule_pep = x[40]\nenzyme_pyk = x[41]\nmolecule_pyruvate = x[42]\ncomplex_enzyme_pyk_2x_metabolite_adp_2x_molecule_pep = x[43]\ncomplex_enzyme_pyk_2x_metabolite_atp_2x_molecule_pyruvate = x[44]\nenzyme_alsS = x[45]\nmolecule_acetolac = x[46]\ncomplex_enzyme_alsS_2x_molecule_pyruvate = x[47]\ncomplex_enzyme_alsS_molecule_acetolac = x[48]\nenzyme_IlvC = x[49]\nmolecule_23dih3mebut = x[50]\ncomplex_enzyme_IlvC_metabolite_nadph_molecule_acetolac = x[51]\ncomplex_enzyme_IlvC_metabolite_nadp_molecule_23dih3mebut = x[52]\nenzyme_IlvD = x[53]\nmolecule_3me2oxo = x[54]\ncomplex_enzyme_IlvD_molecule_23dih3mebut = x[55]\ncomplex_enzyme_IlvD_molecule_3me2oxo = x[56]\nenzyme_kivD = x[57]\nmolecule_isobutanal = x[58]\ncomplex_enzyme_kivD_molecule_3me2oxo = x[59]\ncomplex_enzyme_kivD_molecule_isobutanal = x[60]\nenzyme_yahk = x[61]\nmolecule_isobutanol = x[62]\ncomplex_enzyme_yahk_metabolite_nadph_molecule_isobutanal = x[63]\ncomplex_enzyme_yahk_metabolite_nadp_molecule_isobutanol = x[64]\nenzyme_atpase = x[65]\ncomplex_enzyme_atpase_metabolite_atp = x[66]\ncomplex_enzyme_atpase_metabolite_adp_metabolite_pi = x[67]\n\nr0 = r0_k * enzyme_hex * metabolite_atp * molecule_glucose\nr0_1 = r0_1_k * complex_enzyme_hex_metabolite_atp_molecule_glucose\nr2 = r2_k * complex_enzyme_hex_metabolite_atp_molecule_glucose\nr3 = r3_k * complex_enzyme_hex_metabolite_adp_molecule_g6p\nr3_1 = r3_1_k * enzyme_hex * molecule_g6p * metabolite_adp\nr5 = r5_k * enzyme_pgi * molecule_g6p\nr5_1 = r5_1_k * complex_enzyme_pgi_molecule_g6p\nr7 = r7_k * complex_enzyme_pgi_molecule_g6p\nr8 = r8_k * complex_enzyme_pgi_molecule_f6p\nr8_1 = r8_1_k * enzyme_pgi * molecule_f6p\nr10 = r10_k * enzyme_pfk * metabolite_atp * molecule_f6p\nr10_1 = r10_1_k * complex_enzyme_pfk_metabolite_atp_molecule_f6p\nr12 = r12_k * complex_enzyme_pfk_metabolite_atp_molecule_f6p\nr13 = r13_k * complex_enzyme_pfk_metabolite_adp_molecule_f16p\nr13_1 = r13_1_k * enzyme_pfk * molecule_f16p * metabolite_adp\nr15 = r15_k * enzyme_ald_tpi * molecule_f16p\nr15_1 = r15_1_k * complex_enzyme_ald_tpi_molecule_f16p\nr17 = r17_k * complex_enzyme_ald_tpi_molecule_f16p\nr18 = r18_k * complex_enzyme_ald_tpi_2x_molecule_g3p\nr18_1 = r18_1_k * enzyme_ald_tpi * pow(molecule_g3p, 2)\nr20 = r20_k * enzyme_gapN * pow(metabolite_nadp, 2) * pow(molecule_g3p, 2)\nr20_1 = r20_1_k * complex_enzyme_gapN_2x_metabolite_nadp_2x_molecule_g3p\nr22 = r22_k * complex_enzyme_gapN_2x_metabolite_nadp_2x_molecule_g3p\nr23 = r23_k * complex_enzyme_gapN_2x_metabolite_nadph_2x_molecule_3pg\nr23_1 = r23_1_k * enzyme_gapN * pow(molecule_3pg, 2) * pow(metabolite_nadph, 2)\nr25 = r25_k * enzyme_mGapDH * metabolite_pi * pow(metabolite_nadp, 2) * pow(molecule_g3p, 2)\nr25_1 = r25_1_k * complex_enzyme_mGapDH_2x_metabolite_nadp_metabolite_pi_2x_molecule_g3p\nr27 = r27_k * complex_enzyme_mGapDH_2x_metabolite_nadp_metabolite_pi_2x_molecule_g3p\nr28 = r28_k * complex_enzyme_mGapDH_2x_metabolite_nadph_molecule_13bpg\nr28_1 = r28_1_k * enzyme_mGapDH * molecule_13bpg * pow(metabolite_nadph, 2)\nr30 = r30_k * enzyme_pgk * metabolite_adp * molecule_13bpg\nr30_1 = r30_1_k * complex_enzyme_pgk_metabolite_adp_molecule_13bpg\nr32 = r32_k * complex_enzyme_pgk_metabolite_adp_molecule_13bpg\nr33 = r33_k * complex_enzyme_pgk_metabolite_atp_2x_molecule_3pg\nr33_1 = r33_1_k * enzyme_pgk * pow(molecule_3pg, 2) * metabolite_atp\nr35 = r35_k * enzyme_pgm * pow(molecule_3pg, 2)\nr35_1 = r35_1_k * complex_enzyme_pgm_2x_molecule_3pg\nr37 = r37_k * complex_enzyme_pgm_2x_molecule_3pg\nr38 = r38_k * complex_enzyme_pgm_2x_molecule_2pg\nr38_1 = r38_1_k * enzyme_pgm * pow(molecule_2pg, 2)\nr40 = r40_k * enzyme_eno * pow(molecule_2pg, 2)\nr40_1 = r40_1_k * complex_enzyme_eno_2x_molecule_2pg\nr42 = r42_k * complex_enzyme_eno_2x_molecule_2pg\nr43 = r43_k * complex_enzyme_eno_2x_molecule_pep\nr43_1 = r43_1_k * enzyme_eno * pow(molecule_pep, 2)\nr45 = r45_k * enzyme_pyk * pow(metabolite_adp, 2) * pow(molecule_pep, 2)\nr45_1 = r45_1_k * complex_enzyme_pyk_2x_metabolite_adp_2x_molecule_pep\nr47 = r47_k * complex_enzyme_pyk_2x_metabolite_adp_2x_molecule_pep\nr48 = r48_k * complex_enzyme_pyk_2x_metabolite_atp_2x_molecule_pyruvate\nr48_1 = r48_1_k * enzyme_pyk * pow(molecule_pyruvate, 2) * pow(metabolite_atp, 2)\nr50 = r50_k * enzyme_alsS * pow(molecule_pyruvate, 2)\nr50_1 = r50_1_k * complex_enzyme_alsS_2x_molecule_pyruvate\nr52 = r52_k * complex_enzyme_alsS_2x_molecule_pyruvate\nr53 = r53_k * complex_enzyme_alsS_molecule_acetolac\nr53_1 = r53_1_k * enzyme_alsS * molecule_acetolac\nr55 = r55_k * enzyme_IlvC * metabolite_nadph * molecule_acetolac\nr55_1 = r55_1_k * complex_enzyme_IlvC_metabolite_nadph_molecule_acetolac\nr57 = r57_k * complex_enzyme_IlvC_metabolite_nadph_molecule_acetolac\nr58 = r58_k * complex_enzyme_IlvC_metabolite_nadp_molecule_23dih3mebut\nr58_1 = r58_1_k * enzyme_IlvC * molecule_23dih3mebut * metabolite_nadp\nr60 = r60_k * enzyme_IlvD * molecule_23dih3mebut\nr60_1 = r60_1_k * complex_enzyme_IlvD_molecule_23dih3mebut\nr62 = r62_k * complex_enzyme_IlvD_molecule_23dih3mebut\nr63 = r63_k * complex_enzyme_IlvD_molecule_3me2oxo\nr63_1 = r63_1_k * enzyme_IlvD * molecule_3me2oxo\nr65 = r65_k * enzyme_kivD * molecule_3me2oxo\nr65_1 = r65_1_k * complex_enzyme_kivD_molecule_3me2oxo\nr67 = r67_k * complex_enzyme_kivD_molecule_3me2oxo\nr68 = r68_k * complex_enzyme_kivD_molecule_isobutanal\nr68_1 = r68_1_k * enzyme_kivD * molecule_isobutanal\nr70 = r70_k * enzyme_yahk * metabolite_nadph * molecule_isobutanal\nr70_1 = r70_1_k * complex_enzyme_yahk_metabolite_nadph_molecule_isobutanal\nr72 = r72_k * complex_enzyme_yahk_metabolite_nadph_molecule_isobutanal\nr73 = r73_k * complex_enzyme_yahk_metabolite_nadp_molecule_isobutanol\nr73_1 = r73_1_k * enzyme_yahk * molecule_isobutanol * metabolite_nadp\nr75 = r75_k * enzyme_atpase * metabolite_atp\nr75_1 = r75_1_k * complex_enzyme_atpase_metabolite_atp\nr77 = r77_k * complex_enzyme_atpase_metabolite_atp\nr78 = r78_k * complex_enzyme_atpase_metabolite_adp_metabolite_pi\nr78_1 = r78_1_k * enzyme_atpase * metabolite_adp * metabolite_pi\n\nfuncs = array([ + (-r0) + (r0_1) + (r3) + (-r3_1),\n       + (-r0) + (r0_1) + (-r10) + (r10_1) + (r33) + (-r33_1) + ((2.0)*r48) + (-(2.0)*r48_1) + (-r75) + (r75_1),\n       + (-r0) + (r0_1),\n       + (r3) + (-r3_1) + (-r5) + (r5_1),\n       + (r3) + (-r3_1) + (r13) + (-r13_1) + (-r30) + (r30_1) + (-(2.0)*r45) + ((2.0)*r45_1) + (r78) + (-r78_1),\n       + (r0) + (-r0_1) + (-r2),\n       + (r2) + (-r3) + (r3_1),\n       + (-r5) + (r5_1) + (r8) + (-r8_1),\n       + (r8) + (-r8_1) + (-r10) + (r10_1),\n       + (r5) + (-r5_1) + (-r7),\n       + (r7) + (-r8) + (r8_1),\n       + (-r10) + (r10_1) + (r13) + (-r13_1),\n       + (r13) + (-r13_1) + (-r15) + (r15_1),\n       + (r10) + (-r10_1) + (-r12),\n       + (r12) + (-r13) + (r13_1),\n       + (-r15) + (r15_1) + (r18) + (-r18_1),\n       + ((2.0)*r18) + (-(2.0)*r18_1) + (-(2.0)*r20) + ((2.0)*r20_1) + (-(2.0)*r25) + ((2.0)*r25_1),\n       + (r15) + (-r15_1) + (-r17),\n       + (r17) + (-r18) + (r18_1),\n       + (-r20) + (r20_1) + (r23) + (-r23_1),\n       + (-(2.0)*r20) + ((2.0)*r20_1) + (-(2.0)*r25) + ((2.0)*r25_1) + (r58) + (-r58_1) + (r73) + (-r73_1),\n       + ((2.0)*r23) + (-(2.0)*r23_1) + ((2.0)*r33) + (-(2.0)*r33_1) + (-(2.0)*r35) + ((2.0)*r35_1),\n       + ((2.0)*r23) + (-(2.0)*r23_1) + ((2.0)*r28) + (-(2.0)*r28_1) + (-r55) + (r55_1) + (-r70) + (r70_1),\n       + (r20) + (-r20_1) + (-r22),\n       + (r22) + (-r23) + (r23_1),\n       + (-r25) + (r25_1) + (r28) + (-r28_1),\n       + (-r25) + (r25_1) + (r78) + (-r78_1),\n       + (r28) + (-r28_1) + (-r30) + (r30_1),\n       + (r25) + (-r25_1) + (-r27),\n       + (r27) + (-r28) + (r28_1),\n       + (-r30) + (r30_1) + (r33) + (-r33_1),\n       + (r30) + (-r30_1) + (-r32),\n       + (r32) + (-r33) + (r33_1),\n       + (-r35) + (r35_1) + (r38) + (-r38_1),\n       + ((2.0)*r38) + (-(2.0)*r38_1) + (-(2.0)*r40) + ((2.0)*r40_1),\n       + (r35) + (-r35_1) + (-r37),\n       + (r37) + (-r38) + (r38_1),\n       + (-r40) + (r40_1) + (r43) + (-r43_1),\n       + ((2.0)*r43) + (-(2.0)*r43_1) + (-(2.0)*r45) + ((2.0)*r45_1),\n       + (r40) + (-r40_1) + (-r42),\n       + (r42) + (-r43) + (r43_1),\n       + (-r45) + (r45_1) + (r48) + (-r48_1),\n       + ((2.0)*r48) + (-(2.0)*r48_1) + (-(2.0)*r50) + ((2.0)*r50_1),\n       + (r45) + (-r45_1) + (-r47),\n       + (r47) + (-r48) + (r48_1),\n       + (-r50) + (r50_1) + (r53) + (-r53_1),\n       + (r53) + (-r53_1) + (-r55) + (r55_1),\n       + (r50) + (-r50_1) + (-r52),\n       + (r52) + (-r53) + (r53_1),\n       + (-r55) + (r55_1) + (r58) + (-r58_1),\n       + (r58) + (-r58_1) + (-r60) + (r60_1),\n       + (r55) + (-r55_1) + (-r57),\n       + (r57) + (-r58) + (r58_1),\n       + (-r60) + (r60_1) + (r63) + (-r63_1),\n       + (r63) + (-r63_1) + (-r65) + (r65_1),\n       + (r60) + (-r60_1) + (-r62),\n       + (r62) + (-r63) + (r63_1),\n       + (-r65) + (r65_1) + (r68) + (-r68_1),\n       + (r68) + (-r68_1) + (-r70) + (r70_1),\n       + (r65) + (-r65_1) + (-r67),\n       + (r67) + (-r68) + (r68_1),\n       + (-r70) + (r70_1) + (r73) + (-r73_1),\n       + (r73) + (-r73_1),\n       + (r70) + (-r70_1) + (-r72),\n       + (r72) + (-r73) + (r73_1),\n       + (-r75) + (r75_1) + (r78) + (-r78_1),\n       + (r75) + (-r75_1) + (-r77),\n       + (r77) + (-r78) + (r78_1)    ])\n\n')

from numpy import *
x,f,P = system.load_ODE_model(num_species, num_params

#compartments
default = 1e-06

#global parameters
r0_k = 15.0
r0_1_k = 1.5
r2_k = 36000.0
r3_k = 15.0
r3_1_k = 1.5
r5_k = 15.0
r5_1_k = 1.5
r7_k = 36000.0
r8_k = 15.0
r8_1_k = 1.5
r10_k = 15.0
r10_1_k = 1.5
r12_k = 36000.0
r13_k = 15.0
r13_1_k = 1.5
r15_k = 15.0
r15_1_k = 1.5
r17_k = 36000.0
r18_k = 15.0
r18_1_k = 1.5
r20_k = 15.0
r20_1_k = 1.5
r22_k = 36000.0
r23_k = 15.0
r23_1_k = 1.5
r25_k = 15.0
r25_1_k = 1.5
r27_k = 36000.0
r28_k = 15.0
r28_1_k = 1.5
r30_k = 15.0
r30_1_k = 1.5
r32_k = 36000.0
r33_k = 15.0
r33_1_k = 1.5
r35_k = 15.0
r35_1_k = 1.5
r37_k = 36000.0
r38_k = 15.0
r38_1_k = 1.5
r40_k = 15.0
r40_1_k = 1.5
r42_k = 36000.0
r43_k = 15.0
r43_1_k = 1.5
r45_k = 15.0
r45_1_k = 1.5
r47_k = 36000.0
r48_k = 15.0
r48_1_k = 1.5
r50_k = 15.0
r50_1_k = 1.5
r52_k = 36000.0
r53_k = 15.0
r53_1_k = 1.5
r55_k = 15.0
r55_1_k = 1.5
r57_k = 36000.0
r58_k = 15.0
r58_1_k = 1.5
r60_k = 15.0
r60_1_k = 1.5
r62_k = 36000.0
r6